![image](https://user-images.githubusercontent.com/61610411/129577251-9d68d59e-6c4b-4ac5-8bf0-7f2535241a31.png)

In [1]:
import torch
print(torch.__version__)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

1.9.0+cu102


In [3]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
                                transforms.Resize(256),
                                transforms.RandomCrop(224), # 오려내기
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
])

trainset = torchvision.datasets.STL10(root='./data', split='train', download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,shuffle=True)

testset = torchvision.datasets.STL10(root='./data', split='test', download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,shuffle=False)


Extracting ./data\stl10_binary.tar.gz to ./data
Files already downloaded and verified


![image](https://user-images.githubusercontent.com/61610411/129573173-fa6b19f0-a45d-4db3-968b-8fdc9cfb2802.png)

In [10]:
import torch
import torch.nn as nn

class VGG_A(nn.Module):
    def __init__(self, num_classes:int=1000, init_weights : bool = True):
        super(VGG_A, self).__init__()
        self.net = nn.Sequential(
            # input channel (RGB : 3)
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 224 -> 112

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 112 -> 56

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 56 -> 28

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 28 -> 14

            
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 14 -> 7
        )
        self.fclayer = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 1000),
            #nn.Softmax(dim=1), # loss인 Cross Entropy Loss에서 softmax를 포함한다

        )

    def forward(self, x:torch.Tensor):
        x = self.net(x)
        x = torch.flatten(x,1)
        x = self.fclayer(x)
        return x

print("Done.")

Done.


In [11]:
import time
vgg_a = VGG_A(num_classes=10)
vgg_a = vgg_a.to(device)

classes = ('airplance','bird','car','cat','deer','dog','horse','monkey','ship','truck')

import torch.optim as optim

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(vgg_a.parameters(), lr=0.001)
start_time = time.time()
for epoch in range(10): # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader,0):
        #get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        #zero the parameter gradients

        optimizer.zero_grad()

        #print(input.shape)
        outputs = vgg_a(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 30 == 29:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss/50))
            running_loss = 0.0

print(time.time()-start_time)
print('Finished Training')

[1,    30] loss: 2.214
[1,    60] loss: 1.428
[1,    90] loss: 1.412
[1,   120] loss: 1.409
[1,   150] loss: 1.398
[2,    30] loss: 1.412
[2,    60] loss: 1.403
[2,    90] loss: 1.402
[2,   120] loss: 1.404
[2,   150] loss: 1.391
[3,    30] loss: 1.392
[3,    60] loss: 1.399
[3,    90] loss: 1.398
[3,   120] loss: 1.391
[3,   150] loss: 1.391
[4,    30] loss: 1.392
[4,    60] loss: 1.391
[4,    90] loss: 1.390
[4,   120] loss: 1.394
[4,   150] loss: 1.393
[5,    30] loss: 1.387
[5,    60] loss: 1.387
[5,    90] loss: 1.390
[5,   120] loss: 1.391
[5,   150] loss: 1.390
[6,    30] loss: 1.391
[6,    60] loss: 1.393
[6,    90] loss: 1.390
[6,   120] loss: 1.390
[6,   150] loss: 1.385
[7,    30] loss: 1.391
[7,    60] loss: 1.390
[7,    90] loss: 1.388
[7,   120] loss: 1.390
[7,   150] loss: 1.386
[8,    30] loss: 1.389
[8,    60] loss: 1.392
[8,    90] loss: 1.388
[8,   120] loss: 1.386
[8,   150] loss: 1.388
[9,    30] loss: 1.388
[9,    60] loss: 1.391
[9,    90] loss: 1.391
[9,   120] 

In [16]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs = vgg_a(images)
        _, predicted = torch.max(outputs,1)
        c = (predicted == labels).squeeze()

        for i in range(4):
            label = labels[i]

            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of airplance :  1 %
Accuracy of  bird :  2 %
Accuracy of   car : 84 %
Accuracy of   cat :  0 %
Accuracy of  deer :  0 %
Accuracy of   dog :  0 %
Accuracy of horse :  4 %
Accuracy of monkey :  0 %
Accuracy of  ship :  0 %
Accuracy of truck :  6 %
